In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from backtesting import Backtest, Strategy
from backtesting.lib import resample_apply
from strategies import System

C:\ProgramData\Anaconda3\lib\site-packages\backtesting-0.1.1.dev2+g76465a4-py3.6.egg\backtesting\_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# define convert for backtest filter
def df_date_convert(dataframe, date='2018-09-01 00:00:00'):

    #convert datetime to date
    data_coin['date'] = pd.to_datetime(dataframe['date'],unit='s')

    #convert columns for backtest
    cols = ['open', 'high', 'low', 'close', 'volume', 'date']
    df_backtest = data_coin[cols]
    df_backtest.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Date']

    # filter dates before date
    df_backtest.set_index('Date', inplace=True)
    df_backtest = df_backtest.loc[date:]
    
    return df_backtest

# met 1 strategie een backtest loop maken met daarin alle BTC pairs. Log metrics per coin.  

In [3]:
with open(r'obj/Coin_Dictionary_BTC_Pairs.pkl', 'rb') as fp:
     coin_dict = pickle.load(fp)

In [4]:
print(coin_dict.keys(), '\n\n')
print(coin_dict['BTC_ETH'].keys())

dict_keys(['BTC_ARDR', 'BTC_BAT', 'BTC_BCH', 'BTC_BCHABC', 'BTC_BCHSV', 'BTC_BCN', 'BTC_BNT', 'BTC_BTS', 'BTC_BURST', 'BTC_CLAM', 'BTC_CVC', 'BTC_DASH', 'BTC_DCR', 'BTC_DGB', 'BTC_DOGE', 'BTC_EOS', 'BTC_ETC', 'BTC_ETH', 'BTC_FCT', 'BTC_FOAM', 'BTC_GAME', 'BTC_GAS', 'BTC_GNT', 'BTC_HUC', 'BTC_KNC', 'BTC_LBC', 'BTC_LOOM', 'BTC_LPT', 'BTC_LSK', 'BTC_LTC', 'BTC_MAID', 'BTC_MANA', 'BTC_NAV', 'BTC_NMC', 'BTC_NMR', 'BTC_NXT', 'BTC_OMG', 'BTC_OMNI', 'BTC_PASC', 'BTC_POLY', 'BTC_PPC', 'BTC_QTUM', 'BTC_REP', 'BTC_SBD', 'BTC_SC', 'BTC_SNT', 'BTC_STEEM', 'BTC_STORJ', 'BTC_STR', 'BTC_STRAT', 'BTC_SYS', 'BTC_VIA', 'BTC_VTC', 'BTC_XCP', 'BTC_XEM', 'BTC_XMR', 'BTC_XPM', 'BTC_XRP', 'BTC_ZEC', 'BTC_ZRX']) 


dict_keys(['300', '900', '1800', '7200', '14400', '86400'])


In [5]:
# coins selecten door remove 
remove_coins = ['BTC_BCH', 'BTC_BCHABC', 'BTC_BCHSV', 'BTC_POLY', 'BTC_FOAM', 'BTC_OMNI', 'BTC_LPT']
coin_pairs = list(set(coin_dict.keys())- set(remove_coins))

# coins selecten via list
coin_pairs = ['BTC_DASH', 'BTC_XMR', 'BTC_ETH', 'BTC_LTC', 'BTC_XRP']

# timeframes: '300', '900', '1800', '7200', '14400', '86400'
time_frame = '14400'

In [6]:
# loop through coins pairs and time frames to backtest and get results
result_dict = {}

for coin_pair in coin_pairs:
    time_frame_dict = {}
    
    data_coin = coin_dict[coin_pair][time_frame]
    
    df_backtest = df_date_convert(data_coin)
        
    # run and get backtest results
    bt = Backtest(df_backtest, System, cash=10000, commission=.0025)
    bt_result = bt.run()
        
    # add backtest to list
    result_dict[coin_pair] = bt_result

# Results

In [18]:
df_results = pd.DataFrame(result_dict).T
df_results.head(30)

,Start,End,Duration,Exposure [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Max. Drawdown [%],Avg. Drawdown [%],...,Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Expectancy [%],SQN,Sharpe Ratio,Sortino Ratio,Calmar Ratio,_strategy
BTC_DASH,2018-09-01 00:00:00,2019-01-24 04:00:00,145 days 04:00:00,0,10000,10000,0,30.5147,-0,NaN,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,System
BTC_XMR,2018-09-01 00:00:00,2019-01-24 04:00:00,145 days 04:00:00,0,10000,10000,0,24.9845,-0,NaN,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,System
BTC_ETH,2018-09-01 00:00:00,2019-01-24 04:00:00,145 days 04:00:00,0.114811,10339.4,10340.6,3.39352,20.2304,-0.012544,NaN,...,3.28173,3.28173,0 days 04:00:00,0 days 04:00:00,NaN,NaN,NaN,NaN,261.617,System
BTC_LTC,2018-09-01 00:00:00,2019-01-24 04:00:00,145 days 04:00:00,0,10000,10000,0,2.26906,-0,NaN,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,System
BTC_XRP,2018-09-01 00:00:00,2019-01-24 04:00:00,145 days 04:00:00,1.49254,10422.3,10917.2,4.22296,83.3437,-4.618,-1.76821,...,-3.20376,1.98448,2 days 00:00:00,1 days 02:00:00,5.18824,0.404433,0.270466,NaN,0.429727,System


In [19]:
# filter where exposure is empty
df_results = df_results[df_results["Exposure [%]"] != 0]
df_results.head(30)

,Start,End,Duration,Exposure [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Max. Drawdown [%],Avg. Drawdown [%],...,Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Expectancy [%],SQN,Sharpe Ratio,Sortino Ratio,Calmar Ratio,_strategy
BTC_ETH,2018-09-01 00:00:00,2019-01-24 04:00:00,145 days 04:00:00,0.114811,10339.4,10340.6,3.39352,20.2304,-0.012544,NaN,...,3.28173,3.28173,0 days 04:00:00,0 days 04:00:00,NaN,NaN,NaN,NaN,261.617,System
BTC_XRP,2018-09-01 00:00:00,2019-01-24 04:00:00,145 days 04:00:00,1.49254,10422.3,10917.2,4.22296,83.3437,-4.618,-1.76821,...,-3.20376,1.98448,2 days 00:00:00,1 days 02:00:00,5.18824,0.404433,0.270466,NaN,0.429727,System


In [20]:
df_results_columns = ['Exposure [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Max. Drawdown [%]', 'Avg. Drawdown [%]', '# Trades', 'Win Rate [%]', 'Best Trade [%]',
       'Worst Trade [%]', 'Avg. Trade [%]', 'Expectancy [%]', 'SQN', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio']
df_results_values = df_results[df_results_columns]



#print averages
df_results_values.mean()

Exposure [%]                 0.803674
Equity Final [$]         10380.823800
Equity Peak [$]          10628.903146
Return [%]                   3.808238
Buy & Hold Return [%]       51.787039
Max. Drawdown [%]           -2.315274
Avg. Drawdown [%]           -1.768211
# Trades                     1.500000
Win Rate [%]                75.000000
Best Trade [%]               5.227228
Worst Trade [%]              0.038984
Avg. Trade [%]               2.633106
Expectancy [%]               5.188244
SQN                          0.404433
Sharpe Ratio                 0.270466
Sortino Ratio                     NaN
Calmar Ratio               131.023425
dtype: float64

In [16]:
# get result maximums
print("maximum values: \n\n", df_results_values.astype('float64').idxmax())
print("\n\n")

# get result minimums
print("minimum values: \n\n", df_results_values.astype('float64').idxmin())


maximum values: 

 Exposure [%]             BTC_XRP
Equity Final [$]         BTC_XRP
Equity Peak [$]          BTC_XRP
Return [%]               BTC_XRP
Buy & Hold Return [%]    BTC_XRP
Max. Drawdown [%]        BTC_ETH
Avg. Drawdown [%]        BTC_XRP
# Trades                 BTC_XRP
Win Rate [%]             BTC_ETH
Best Trade [%]           BTC_XRP
Worst Trade [%]          BTC_ETH
Avg. Trade [%]           BTC_ETH
Expectancy [%]           BTC_XRP
SQN                      BTC_XRP
Sharpe Ratio             BTC_XRP
Sortino Ratio                NaN
Calmar Ratio             BTC_ETH
dtype: object



minimum values: 

 Exposure [%]             BTC_ETH
Equity Final [$]         BTC_ETH
Equity Peak [$]          BTC_ETH
Return [%]               BTC_ETH
Buy & Hold Return [%]    BTC_ETH
Max. Drawdown [%]        BTC_XRP
Avg. Drawdown [%]        BTC_XRP
# Trades                 BTC_ETH
Win Rate [%]             BTC_XRP
Best Trade [%]           BTC_ETH
Worst Trade [%]          BTC_XRP
Avg. Trade [%]       

# check individuele gevallen

In [10]:
coin = 'BTC_REP'
data_coin = coin_dict[coin][time_frame]
df_backtest = df_date_convert(data_coin)

# run and get backtest results
bt = Backtest(df_backtest, System, cash=10000, commission=.0025)
bt.run()      


Start                     2018-09-01 00:00:00
End                       2019-01-24 04:00:00
Duration                    145 days 04:00:00
Exposure [%]                                0
Equity Final [$]                        10000
Equity Peak [$]                         10000
Return [%]                                  0
Buy & Hold Return [%]                 42.3008
Max. Drawdown [%]                          -0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                            NaN
Max. Trade Duration                       NaT
Avg. Trade Duration                       NaT
Expectancy [%]                            NaN
SQN                                       NaN
Sharpe Ratio                      

In [11]:
bt.plot()